<a href="https://colab.research.google.com/github/OluwoleDove/medical_image_classifier/blob/main/Pytorch_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using the GPU!")
    device = torch.device("cuda")
else:
    print("CUDA is not available. PyTorch will use the CPU.")
    device = torch.device("cpu")

CUDA is available. PyTorch is using the GPU!


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Load data into DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16361021.35it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 496818.16it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4509344.45it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4854874.81it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
# Initialize the network and move it to GPU if available
model = Net().to(device)

In [6]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # Move data to GPU
        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [8]:
def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # Move data to GPU
            output = model(data)
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [9]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308317
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.145975
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.106815
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.198163
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.155490
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.050165
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.162102
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.036709
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.032853
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.012841

Test set: Average loss: 0.0000, Accuracy: 9842/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.105967
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.019798
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.109971
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.070471
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.043568
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.002676
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.144793
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.094404
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.076143
T

In [10]:
torch.save(model.state_dict(), "mnist_cnn.pth")

In [11]:
#TO LOAD THE MODEL LATER
model = Net().to(device)
model.load_state_dict(torch.load("mnist_cnn.pth"))

<ipython-input-11-668c86614cb1>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mnist_cnn.pth"))


<All keys matched successfully>